# import

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.transforms import autoaugment, transforms
from torch.utils.data import DataLoader, random_split, ConcatDataset, Subset
from tqdm import tqdm
import models.optimizer.optimizer as optimizer
from torch.optim.lr_scheduler import ReduceLROnPlateau
from helper.training_early_stop import EarlyStop
import helper.utility as utility

# select device

In [2]:
device = utility.select_devices(use_cudnn_if_avaliable=True)

using CUDA + cudnn


# Data Pre-processing

### transformer parameter

In [3]:
# Alex 227, VGG 224, ResNet 224, Unet 96
size = (64, 64)
batch_size = 16
channels = 1

data_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=channels), 
    transforms.Resize(size),
    transforms.ToTensor()
])

### create dataloader

#### dataloader 1 RAF-DB

In [4]:
test_path = '../RAF-DB/test'
test_dataset1 = datasets.ImageFolder(test_path, transform=data_transforms)
test_loader = DataLoader(test_dataset1, batch_size=batch_size, shuffle=False, pin_memory=False, num_workers=16)

In [5]:
test_l1 = test_loader
test_l2 = test_loader
test_l3 = test_loader

In [6]:
best_result = {}
final_result = {}
loss_result = {}

best_result = utility.read_pickle_files('./model_data/test_result/best_result.pkl')
final_result = utility.read_pickle_files('./model_data/test_result/final_result.pkl')
loss_result = utility.read_pickle_files('./model_data/test_result/loss_result.pkl')

In [11]:
print(best_result)
print(final_result)
print(loss_result)

{'custom_v0_Adam_[RAF(aug12)]': 0.7327249022164276, 'custom_v1_Adam_[RAF(aug12)]': 0.7434810951760105, 'custom_v2_Adam_[RAF(aug12)]': 0.7428292046936115, 'custom_v3_Adam_[RAF(aug12)]': 0.7884615384615384, 'custom_v4_Adam_[RAF(aug12)]': 0.7796610169491526, 'custom_v5_Adam_[RAF(aug12)]': 0.7865058670143416, 'custom_v6.1_Adam_[RAF(aug12)]': 0.8021512385919165, 'custom_v6_Adam_[RAF(aug12)]': 0.7940026075619296, 'custom_v7.2_Adam_[RAF(aug12)]': 0.8041069100391134, 'custom_v7_Adam_[RAF(aug12)]': 0.7969361147327249, 'dcnn_v0+_Adam_[RAF(aug12)]': 0.7676010430247718, 'dcnn_v0_Adam_[RAF(aug12)]': 0.7584745762711864, 'dcnn_v1_Adam_[RAF(aug12)]': 0.7627118644067796, 'dcnn_v2_Adam_[RAF(aug12)]': 0.7509778357235984, 'dcnn_v3+_Adam_[RAF(aug12)]': 0.7803129074315515, 'dcnn_v3_Adam_[RAF(aug12)]': 0.7780312907431551, 'dcnn_v4_Adam_[RAF(aug12)]': 0.7884615384615384, 'dcnn_v5_Adam_[RAF(aug12)]': 0.7985658409387223, 'dcnn_v6_Adam_[RAF(aug12)]': 0.8083441981747066, 'dcnn_v7_Adam_[RAF(aug12)]': 0.79628422425

### Test Accuracy

In [8]:
from models.new_models import DCNN_v7 as m
test_name = 'v7_Adam_[RAF(aug12)]'

key_name = 'dcnn_'+test_name

model = m.EmotionCNN(num_classes=7, input_channel=channels)

pth = './model_data/DCNN/'
pth_save_path = pth + test_name + '/model.pth'
pth_manual_save_path = pth + test_name + '/manual_save_model.pth'
pth_save_path_loss = pth + test_name + '/best_loss_model.pth'
save_place = './model_data/test_result/'

# # evaluate model
# result = utility.model_validation(model, device, test_l1, pth_save_path, save_place, file_name='1', ext=False,show=False)
# best_result[key_name] = result
# result = utility.model_validation(model, device, test_l2, pth_manual_save_path, save_place, file_name='2', ext=False,show=False)
# final_result[key_name] = result
# result = utility.model_validation(model, device, test_l3, pth_save_path_loss, save_place, file_name='3', ext=False,show=False)
# loss_result[key_name] = result

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 32, 64, 64]          320
├─BatchNorm2d: 1-2                       [-1, 32, 64, 64]          64
├─ReLU: 1-3                              [-1, 32, 64, 64]          --
├─Conv2d: 1-4                            [-1, 64, 64, 64]          18,496
├─BatchNorm2d: 1-5                       [-1, 64, 64, 64]          128
├─Conv2d: 1-6                            [-1, 64, 64, 64]          640
├─BatchNorm2d: 1-7                       [-1, 64, 64, 64]          128
├─ReLU: 1-8                              [-1, 64, 64, 64]          --
├─MaxPool2d: 1-9                         [-1, 64, 32, 32]          --
├─Dropout: 1-10                          [-1, 64, 32, 32]          --
├─Conv2d: 1-11                           [-1, 64, 32, 32]          36,928
├─BatchNorm2d: 1-12                      [-1, 64, 32, 32]          128
├─ReLU: 1-13                             [-1, 64, 32, 32]          --
├─

In [9]:
# utility.save_pickle_files(best_result, './model_data/test_result/best_result.pkl')
# print(best_result)
# utility.save_pickle_files(final_result, './model_data/test_result/final_result.pkl')
# print(final_result)
# utility.save_pickle_files(loss_result, './model_data/test_result/loss_result.pkl')
# print(loss_result)